In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import shutil
import pandas as pd

print(tf.__version__)
tf.logging.set_verbosity(tf.logging.INFO)

# List the CSV columns
CSV_COLUMNS = ['fare_amount', 'pickup_datetime','pickup_longitude','pickup_latitude',
               'dropoff_longitude','dropoff_latitude', 'passenger_count', 'key']

#Choose which column is your label
LABEL_COLUMN = 'fare_amount'
import os
print(os.listdir("../input"))

#TODO create two separate datasets for Training and Evaluation

In [ ]:
#This is just to have a look at the data
PATH = '../input'
train_df = pd.read_csv(f'{PATH}/train.csv', nrows=10000)
train_df['distance'] = np.sqrt(np.abs(train_df['pickup_longitude']-train_df['dropoff_longitude'])**2 +
                        np.abs(train_df['pickup_latitude']-train_df['dropoff_latitude'])**2)
train_df.head()
train_df.describe()

In [ ]:
BATCH_SIZE=8
dataset = tf.contrib.data.make_csv_dataset(
    file_pattern=f'{PATH}/train.csv',
    batch_size=BATCH_SIZE,
    column_names=None,
    column_defaults=None,
    label_name='fare_amount',
    select_columns=[1, 2, 3, 4, 5, 6, 7],
    field_delim=',',
    use_quote_delim=True,
    na_value='',
    header=True,
    num_epochs=None,
    shuffle=True,
    shuffle_buffer_size=10000,
    shuffle_seed=None,
    prefetch_buffer_size=1,
    num_parallel_reads=1,
    num_parallel_parser_calls=2,
    sloppy=False,
    num_rows_for_inference=100
)

In [ ]:
next_element = dataset.make_one_shot_iterator().get_next()
with tf.Session() as sess:
    features, label = sess.run(next_element)
    print("Features:\n", features, "\n\nLabel:\n", label)


In [ ]:
def pd_weekDay(year, month, day):
    df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
    date_df = pd.to_datetime(df)
    return date_df.dt.weekday.astype(np.int32)

years=np.array([2018, 2018, 2018])
months=np.array([8, 11, 1])
days=np.array([20, 6, 8])
print(pd_weekDay(years, months, days))


In [ ]:
def tf_isAirport(latitude,longitude,airport_name='JFK'):
    jfkcoord = tf.constant([-73.8352, -73.7401, 40.6195, 40.6659])
    if airport_name=='JFK':
        coord = jfkcoord
    is_airport = \
    tf.logical_and(
        tf.logical_and(
            tf.greater(latitude, coord[0]), tf.less(latitude, coord[1])
        ),
        tf.logical_and(
            tf.greater(longitude, coord[2]), tf.less(longitude, coord[3])
        )
    )
    return is_airport

In [ ]:
def feat_eng_func(features, label=None):
    print("Feature Engineered Label:", label)
    #New features based on pickup datetime
    features['pickup_year'] = tf.string_to_number(tf.substr(features['pickup_datetime'], 0, 4), tf.int32)
    features['pickup_month'] = tf.string_to_number(tf.substr(features['pickup_datetime'], 5, 2), tf.int32)
    features['pickup_day'] = tf.string_to_number(tf.substr(features['pickup_datetime'], 8, 2), tf.int32)
    features['pickup_hour'] = tf.string_to_number(tf.substr(features['pickup_datetime'], 11, 2), tf.int32)
    features['pickup_weekday'] = tf.py_func(pd_weekDay,
                                            [features['pickup_year'], features['pickup_month'], features['pickup_day']],
                                            tf.int32,
                                            stateful=False,
                                            name='Weekday'
                                           )
    #Normalize year and add decimals for months. This is because fares increase with time
    features['pickup_dense_year'] = (
                tf.cast(features['pickup_year'], tf.float32) + \
                tf.cast(features['pickup_month'], tf.float32) / tf.constant(12.0, tf.float32) -  \
                 tf.constant(2009.0, tf.float32) ) /  \
                 tf.constant(6.0, tf.float32) 
   
    #Clip latitudes and longitudes
    minlat = tf.constant(40.0)
    maxlat = tf.constant(42.0)
    minlon = tf.constant(-75.0)
    maxlon = tf.constant(-72.0)
    features['pickup_longitude'] = tf.clip_by_value(features['pickup_longitude'], minlon, maxlon)
    features['pickup_latitude'] = tf.clip_by_value(features['pickup_latitude'], minlat, maxlat)
    features['dropoff_longitude'] = tf.clip_by_value(features['dropoff_longitude'], minlon, maxlon)
    features['dropoff_latitude'] = tf.clip_by_value(features['dropoff_latitude'], minlat, maxlat)
    #TODO feature for the day of the week
    #New features based on pickup and dropoff position
    features['longitude_dist'] = tf.abs(features['pickup_longitude'] - features['dropoff_longitude'])
    features['latitude_dist'] = tf.abs(features['pickup_latitude'] - features['dropoff_latitude'])
    #compute euclidean distance of the trip
    features['distance'] = tf.sqrt(features['longitude_dist']**2 + features['latitude_dist']**2)
    long_distance = tf.constant(0.07)
    features['is_long_distance'] = tf.less(long_distance, features['distance'])
    features['is_JFK_pickup'] = tf_isAirport(features['pickup_latitude'], 
                                             features['pickup_longitude'],
                                             airport_name='JFK')
    features['is_JFK_dropoff'] = tf_isAirport(features['dropoff_latitude'], 
                                             features['dropoff_longitude'],
                                             airport_name='JFK')
#    features['pickup_minute'] = tf.substr(features['pickup_datetime'], 14, 2)
#TODO normalize long and lat
#TODO remove outliers on passenger_count and fare_amount
    print(features)
    if label == None:
        return features
    return (features, label)

In [ ]:
# Create an input function that stores your data into a dataset
def read_dataset(filename, mode, batch_size = 512):
    def _input_fn():    
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            shuffle = True
        else:
            num_epochs = 1 # end-of-input after this
            shuffle = False

        if mode == tf.estimator.ModeKeys.PREDICT:
            label_name=None
            select_columns=[1, 2, 3, 4, 5, 6]
        else:
            label_name ='fare_amount'
            select_columns = [1, 2, 3, 4, 5, 6, 7]

        # Create list of files that match pattern
        file_list = tf.gfile.Glob(filename)
        # Create Dataset from the CSV files
        dataset = tf.contrib.data.make_csv_dataset(
            file_pattern=file_list,
            batch_size=batch_size,
            column_names=None,
            column_defaults=None,
            label_name=label_name,
            select_columns=select_columns,
            field_delim=',',
            use_quote_delim=True,
            na_value='',
            header=True,
            num_epochs=num_epochs,
            shuffle=shuffle,
            shuffle_buffer_size=128*batch_size,
            shuffle_seed=None,
            prefetch_buffer_size=1,
            num_parallel_reads=1,
            num_parallel_parser_calls=3,
            sloppy=False,
            num_rows_for_inference=100
        )
        train_lines = 55000000 // batch_size // 100 * 80
        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = dataset.take(train_lines)
        elif mode == tf.estimator.ModeKeys.EVAL:
            dataset = dataset.skip(train_lines) #EVAL on different data

        dataset = dataset.map(feat_eng_func)
        return dataset.make_one_shot_iterator().get_next()
    return _input_fn


In [ ]:
train_input_fn = read_dataset(f'{PATH}/train.csv', tf.estimator.ModeKeys.TRAIN, batch_size = 8)
next_element = train_input_fn()



In [ ]:
with tf.Session() as sess:
    features, label = sess.run(train_input_fn())
    print("Features:\n", features, "\n\nLabel:\n", label)

In [ ]:
# Define your feature columns
def create_feature_cols():
    hour_cat = tf.feature_column.categorical_column_with_identity('pickup_hour', 24 )
    weekday_cat = tf.feature_column.categorical_column_with_identity('pickup_weekday', 7)
    hour_X_weekday = tf.feature_column.crossed_column([hour_cat, weekday_cat], 500)

    return [
    tf.feature_column.numeric_column('pickup_longitude'),
    tf.feature_column.numeric_column('pickup_latitude'),
    tf.feature_column.numeric_column('dropoff_longitude'),
    tf.feature_column.numeric_column('dropoff_latitude'),
    tf.feature_column.numeric_column('passenger_count'),
    #TODO use pickup_year
    tf.feature_column.numeric_column('pickup_dense_year'),
#    tf.feature_column.numeric_column('pickup_year'),
#    tf.feature_column.numeric_column('pickup_month'),
#    tf.feature_column.numeric_column('pickup_day'),
    #TODO use embeddings for the hour
    #tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('pickup_hour', (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    #                                                                        11, 12, 13, 14, 15, 16, 17, 18,
    #                                                                         19, 20, 21, 22, 23) )
    #                                  ),
    #tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('pickup_weekday', (0, 1, 2, 3, 4, 5, 6)
    #                                                                                            )),
    tf.feature_column.embedding_column(hour_X_weekday, 2),
    tf.feature_column.numeric_column('longitude_dist'),
    tf.feature_column.numeric_column('latitude_dist'),
    tf.feature_column.numeric_column('distance'),
    tf.feature_column.numeric_column('is_JFK_pickup'),
    tf.feature_column.numeric_column('is_JFK_dropoff'),
#    tf.feature_column.numeric_column('is_long_distance')
  ]

In [ ]:
BATCH_SIZE = 512
train_input_fn = read_dataset(f'{PATH}/train.csv', tf.estimator.ModeKeys.TRAIN, batch_size = BATCH_SIZE)
eval_input_fn = read_dataset(f'{PATH}/train.csv', tf.estimator.ModeKeys.EVAL, batch_size = BATCH_SIZE)
# Create estimator train and evaluate function
def train_and_evaluate(output_dir, num_train_steps):
#    estimator = tf.estimator.LinearRegressor(model_dir = output_dir, feature_columns = create_feature_cols())
    estimator = tf.estimator.DNNRegressor(model_dir = output_dir, feature_columns = create_feature_cols(),
                                         hidden_units=[32, 32, 16])
    train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn, 
                                      max_steps = num_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn, 
                                    steps = None, 
                                    start_delay_secs = 1, # start evaluating after N seconds, 
                                    throttle_secs = 60)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    return estimator
    

OUTDIR = './trained_model'
shutil.rmtree(OUTDIR, ignore_errors = True)
#model = train_and_evaluate(OUTDIR, NUM_STEPS)
#print("BATCH SIZE = ", BATCH_SIZE,"\nDataset Take = ", 128*BATCH_SIZE)


In [ ]:
BATCH_SIZE = 256
OUTDIR = './trained_model'
train_input_fn = read_dataset(f'{PATH}/train.csv', tf.estimator.ModeKeys.TRAIN, batch_size = BATCH_SIZE)
eval_input_fn = read_dataset(f'{PATH}/train.csv', tf.estimator.ModeKeys.EVAL, batch_size = BATCH_SIZE)
shutil.rmtree(OUTDIR, ignore_errors = True)
#estimator = tf.estimator.LinearRegressor(model_dir = OUTDIR, feature_columns = create_feature_cols())
estimator = tf.estimator.DNNRegressor(model_dir = OUTDIR, feature_columns = create_feature_cols(),
                                     hidden_units=[128, 64, 32])
estimator.train(train_input_fn, max_steps=40000)
estimator.evaluate(eval_input_fn, name='train_eval')

In [ ]:
predict_input_fn = read_dataset(f'{PATH}/test.csv', tf.estimator.ModeKeys.PREDICT, batch_size=1)
predictions = estimator.predict(predict_input_fn)

s = pd.Series()
for i, p in enumerate(predictions):
    if i < 10000:
        s.at[i] = p['predictions'][0]
    else:
        break
s.describe()
s.to_csv("DNNregr.csv")
#    print("Prediction %s: %s" % (i + 1, p))

In [ ]:
def weekDay(year, month, day):
    offset = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]
    week   = ['Sunday', 
              'Monday', 
              'Tuesday', 
              'Wednesday', 
              'Thursday',  
              'Friday', 
              'Saturday']
    afterFeb = 1
    if month > 2: afterFeb = 0
    aux = year - 1700 - afterFeb
    # dayOfWeek for 1700/1/1 = 5, Friday
    dayOfWeek  = 5
    # partial sum of days betweem current date and 1700/1/1
    dayOfWeek += (aux + afterFeb) * 365                  
    # leap year correction    
    dayOfWeek += aux // 4 - aux // 100 + (aux + 100) // 400     
    # sum monthly and day offsets
    dayOfWeek += offset[month - 1] + (day - 1)               
    dayOfWeek %= 7
    return dayOfWeek, week[dayOfWeek]

print(weekDay(2018, 8, 17))

In [ ]:
nyc_airports={'JFK':{'min_lng':-73.8352,
     'min_lat':40.6195,
     'max_lng':-73.7401, 
     'max_lat':40.6659},
              
    'EWR':{'min_lng':-74.1925,
            'min_lat':40.6700, 
            'max_lng':-74.1531, 
            'max_lat':40.7081

        },
    'LaGuardia':{'min_lng':-73.8895, 
                  'min_lat':40.7664, 
                  'max_lng':-73.8550, 
                  'max_lat':40.7931
        
    }
    
}

def isAirport(latitude,longitude,airport_name='JFK'):
    if latitude>=nyc_airports[airport_name]['min_lat'] and 
        latitude<=nyc_airports[airport_name]['max_lat'] and 
        longitude>=nyc_airports[airport_name]['min_lng'] and 
        longitude<=nyc_airports[airport_name]['max_lng']:
        return 1
    else:
        return 0




In [ ]:
def tf_isAirport(latitude,longitude,airport_name='JFK'):
    jfkcoord = tf.constant([-73.8352, -73.7401, 40.6195, 40.6659])
    if airport_name=='JFK':
        coord = jfkcoord
    is_airport = \
    tf.logical_and(
        tf.logical_and(
            tf.greater(latitude, coord[0]), tf.less(latitude, coord[1])
        ),
        tf.logical_and(
            tf.greater(longitude, coord[2]), tf.less(longitude, coord[3])
        )
    )
    return is_airport

isair = tf_isAirport(-73.8342, 40.62)
with tf.Session() as sess:
    output = sess.run(isair)
    print(output)